# 네트워크 분석 따라해보기;;;

# 대분류, 중분류를 구매 패턴을 통해서 무언가 발견할 수 있지 않을까 에 대한 의문

- 유저아이디, 오더아이디 기준, add_to_cart_order 순서에 따른 엣지 생성 -> 노드화

## 1. 임포트 하기 / 전처리 하기 : 필요한 것들만

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
orders_df = pd.read_csv('orders.csv', dtype = 
                        {'order_id' : np.int32, 'user_id' : np.int32,
                        'eval_set' : 'category',
                        'order_number' : np.int16,
                        'order_dow' : np.int8,
                        'order_hour_of_day' : np.int8,
                        'days_since_prior_order' : np.float32})

products = pd.read_csv('products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8})

print('loading prior')
priors = pd.read_csv('order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv('order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print ('loading')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

loading prior
loading train
loading


In [3]:
orders = orders_df[orders_df['eval_set']=='prior']

In [4]:
orders = orders.drop('eval_set', axis = 1)

In [5]:
orders.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [6]:
df_one = pd.merge(priors, products, on = 'product_id')

In [7]:
df_two = pd.merge(df_one, orders, on = 'order_id')

In [8]:
df_two.head(5)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,86,16,202279,3,5,9,8.0
1,2,28985,2,1,Michigan Organic Kale,83,4,202279,3,5,9,8.0
2,2,9327,3,0,Garlic Powder,104,13,202279,3,5,9,8.0
3,2,45918,4,1,Coconut Butter,19,13,202279,3,5,9,8.0
4,2,30035,5,0,Natural Sweetener,17,13,202279,3,5,9,8.0


In [12]:
df_two = pd.merge(df_two, aisles, on = 'aisle_id')

In [15]:
df_two = pd.merge(df_two, departments, on ='department_id')

In [16]:
df_two

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle,department
0,2,33120,1,1,Organic Egg Whites,86,16,202279,3,5,9,8.0,eggs,dairy eggs
1,26,33120,5,0,Organic Egg Whites,86,16,153404,2,0,16,7.0,eggs,dairy eggs
2,120,33120,13,0,Organic Egg Whites,86,16,23750,11,6,8,10.0,eggs,dairy eggs
3,327,33120,5,1,Organic Egg Whites,86,16,58707,21,6,9,8.0,eggs,dairy eggs
4,390,33120,28,1,Organic Egg Whites,86,16,166654,48,0,12,9.0,eggs,dairy eggs
5,537,33120,2,1,Organic Egg Whites,86,16,180135,15,2,8,3.0,eggs,dairy eggs
6,582,33120,7,1,Organic Egg Whites,86,16,193223,6,2,19,10.0,eggs,dairy eggs
7,608,33120,5,1,Organic Egg Whites,86,16,91030,11,3,21,12.0,eggs,dairy eggs
8,623,33120,1,1,Organic Egg Whites,86,16,37804,63,3,12,3.0,eggs,dairy eggs
9,689,33120,4,1,Organic Egg Whites,86,16,108932,16,1,13,3.0,eggs,dairy eggs


## 2. 네트워크 데이터 셋 만들기

네트워크를 헤비, 미들, 로우 셀러로 구분지어서 분석 하여 보자.
못함

### 2-1. 내가 찾고자 하는 관계를 칼럼값으로 뽑아주면 됨
    - department / aisle / product_name

In [43]:
df_three = df_two[['order_id', 'add_to_cart_order', 'user_id', 'product_name', 'aisle', 'department']]

In [45]:
df_three.sort_values(['order_id', 'add_to_cart_order'], inplace = True)
# 내가 보고싶은 네트워크 데이터 프레임.
# 대분류별, 중분류별, 네임별로 뽑아서 보면 됨

C:\python3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [46]:
df_three

,order_id,add_to_cart_order,user_id,product_name,aisle,department
0,2,1,202279,Organic Egg Whites,eggs,dairy eggs
5414016,2,2,202279,Michigan Organic Kale,fresh vegetables,produce
14893307,2,3,202279,Garlic Powder,spices seasonings,pantry
15105399,2,4,202279,Coconut Butter,oils vinegars,pantry
15350865,2,5,202279,Natural Sweetener,baking ingredients,pantry
5414017,2,6,202279,Carrots,fresh vegetables,produce
15677557,2,7,202279,Original Unflavored Gelatine Mix,doughs gelatins bake mixes,pantry
15783646,2,8,202279,All Natural No Stir Creamy Almond Butter,spreads,pantry
8832037,2,9,202279,Classic Blend Cole Slaw,packaged vegetables fruits,produce
2470592,3,1,205970,Total 2% with Strawberry Lowfat Greek Strained...,yogurt,dairy eggs


In [16]:
#df_five = pd.merge(df_four, departments, on = 'department_id')

In [18]:
#df_five.sort_values(['order_id', 'add_to_cart_order'], inplace = True)

## 3. 인자값을 받아서 네트워크 그래프를 뽑아내는 함수 형성
    - 인자는 내가 원하는 종류, 범위(갯수)

In [47]:
def ask_size() :
    print ('''choose what you want and size, remember, first parameter is kind and second is size.
           1 is departmaet, 2 is aisle, 3 is product name and size means loc[size], 
           so do not enter huge size, maybe your computer will shut down''' )
    try:
        kind = int(input('1 dep or 2 aisle or 3 name'))
        size = input('size')
    except:
        print ('you entered wrong number')
    return network(kind, size)
# 종류를 선택하는 함수

def network(kind, size):
    df_five_exe = df_three[:int(size)]
    exe = {}
    if kind == 1:
        kind = 'department'
    elif kind == 2:
        kind = 'aisle'
    else:
        kind ='product_name'
    lst_c = list(set(df_five_exe[kind].values))
    print ('loading...wait')
    for name, group in df_five_exe[kind].groupby(df_five_exe['order_id']):
        exe[name] = group.values
    
    return make_net(exe, lst_c, kind, size)
# 종류와 갯수를 받아서 준비하는 함수
    
def make_net(argu, lst, kind, size):
    pra = []
    for i in (argu):
        num = len(argu[i])
        for n in range(num):
            if n != num - 1: 
                pra.append([argu[i][n], argu[i][n+1]])
            else:
                pass
    return make_nett(pra, lst, kind, size)
# 해당 종류를 딕트화 하는 함수

def make_nett(fun, lst, kind, size):
    import networkx as nx
    G = nx.MultiGraph()
    G.add_nodes_from(lst)
    G.add_edges_from(fun)
    print ('import is end')
    
    with open(("{} and {}.graphml").format(kind,size), "wb") as graph:
        nx.write_graphml(G, graph)
    print (('{} and {} graph was completed').format(kind,size))
    print ('node is', len(G.nodes()), 'edges are', len(G.edges()))
           
# 해당 조건에 따른 네트워크 파일을 만드는 함수



In [49]:
ask_size()

choose what you want and size, remember, first parameter is kind and second is size.
           1 is departmaet, 2 is aisle, 3 is product name and size means loc[size], 
           so do not enter huge size, maybe your computer will shut down
1 dep or 2 aisle or 3 name2
size10000
loading...wait
import is end
aisle and 10000 graph was completed
node is 133 edges are 9023
